## Ebola infected Macaque Sample Composition Trajectory Identification

```
Indices:

- c cell type
- g genes
- m samples
- k deformation polynomial degree
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from time_deconv.time_deconv import *

## Parameter Configuration

In [ ]:
device = torch.device("cuda:0")
dtype = torch.float32
dtype_np = np.float32

## Train Model

In [ ]:
bulk_anndata_path = "/home/nbarkas/disk2/deconvolution_method/datasets/ebov/load_data_python/ebov_bulk.h5ad"
sc_anndata_path = "/home/nbarkas/disk2/deconvolution_method/datasets/ebov/load_data_python/ebov_sc.h5ad"

In [ ]:
with open(bulk_anndata_path, 'rb') as fh:
    bulk_anndata  = anndata.read_h5ad(fh)
with open(sc_anndata_path, 'rb') as fh:
    sc_anndata = anndata.read_h5ad(fh)

In [ ]:
# select samples only after or on tp 0
bulk_anndata = bulk_anndata[bulk_anndata.obs['dpi_time'] >= 0,]

In [ ]:
ebov_dataset = DeconvolutionDataset(
    sc_anndata = sc_anndata,
    sc_celltype_col = "Subclustering_reduced",
    bulk_anndata = bulk_anndata,
    bulk_time_col = "dpi_time",
    dtype_np = dtype_np,
    dtype = dtype,
    device=device,
    feature_selection_method = 'overdispersed_bulk_and_high_sc' #'overdispersed_bulk'
)

In [ ]:
pseudo_time_reg_deconv = TimeRegularizedDeconvolution(
    dataset=ebov_dataset,
    polynomial_degree = 20,
    basis_functions = "polynomial",
    device=device,
    dtype=dtype)

In [ ]:
pseudo_time_reg_deconv.fit_model(n_iters=5_000, verbose=True, log_frequency=1000)

## Examine Outputs

In [ ]:
pseudo_time_reg_deconv.plot_loss()

In [ ]:
pseudo_time_reg_deconv.calculate_composition_trajectories(n_intervals = 1000)

In [ ]:
pseudo_time_reg_deconv.plot_composition_trajectories()

In [ ]:
pseudo_time_reg_deconv.plot_phi_g_distribution()

In [ ]:
pseudo_time_reg_deconv.plot_beta_g_distribution()
matplotlib.pyplot.yscale('log')

# Synthetic Data

In [ ]:
batch_dict = generate_batch(pseudo_time_reg_deconv.dataset, device, dtype)

## Evaluate Simulation 1 (100 samples; polynomial)

In [ ]:
# Simulate and plot proportions
sim_res = simulate_with_sigmoid_proportions(
    num_samples=100, 
    reference_deconvolution=pseudo_time_reg_deconv,
    dirichlet_alpha = 1e10,
)


plot_simulated_proportions(sim_res)
matplotlib.pyplot.show()

In [ ]:
simulated_bulk = generate_anndata_from_sim(sim_res, reference_deconvolution=pseudo_time_reg_deconv)

In [ ]:
ebov_simulated_dataset = DeconvolutionDataset(
    sc_anndata = sc_anndata,
    sc_celltype_col = "Subclustering_reduced",
    bulk_anndata = simulated_bulk,
    bulk_time_col = "time",
    dtype_np = dtype_np,
    dtype = dtype,
    device = device,
    feature_selection_method = 'common' 
)

In [ ]:
pseudo_time_reg_deconv_sim = TimeRegularizedDeconvolution(
    dataset=ebov_simulated_dataset,
    polynomial_degree = 20,
    basis_functions = "polynomial",
    device=device,
    dtype=dtype)

In [ ]:
pseudo_time_reg_deconv_sim.fit_model(n_iters=5_001, verbose=True, log_frequency=1000)

In [ ]:
pseudo_time_reg_deconv_sim.plot_loss()

In [ ]:
pseudo_time_reg_deconv_sim.calculate_composition_trajectories(n_intervals = 1000)

In [ ]:
plot_simulated_proportions(sim_res)
matplotlib.pyplot.show()

In [ ]:
pseudo_time_reg_deconv_sim.plot_composition_trajectories()

In [ ]:
calculate_prediction_error(sim_res, pseudo_time_reg_deconv_sim)